# 🎲 Monte Carlo Lineup Simulations

## Overview
Simulates 50,000 fantasy football outcomes using lognormal distributions based on player projections.

## Methodology
- **Distribution**: Lognormal
- **Simulations**: N = 10,000
- **Seed**: 1738 (for reproducibility)

## Lognormal Parameterization
Given player mean (μ) and standard deviation (σ):

$$\phi = \sqrt{\sigma^2 + \mu^2}$$

$$\sigma_{ln} = \sqrt{\log\left(\frac{\phi}{\mu}\right)^2}$$

$$\mu_{ln} = \log\left(\frac{\mu^2}{\phi}\right)$$

For degenerate cases (σ = 0), we set $\sigma_{ln} = 10^{-9}$


## 📦 Setup & Configuration


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import json
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

# Set style for elegant plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 10

# Simulation parameters
SEED = 1738
N_SIMULATIONS = 50000
CURRENT_WEEK = 10

np.random.seed(SEED)

print("✓ Libraries loaded")
print(f"✓ Random seed: {SEED}")
print(f"✓ Simulations: {N_SIMULATIONS:,}")
print(f"✓ Week: {CURRENT_WEEK}")


✓ Libraries loaded
✓ Random seed: 1738
✓ Simulations: 50,000
✓ Week: 10


## 📊 Load Lineup Data


In [ ]:
# Load team lineups from optimizer output
df_lineups = pd.read_csv(f'../data/csv/team_lineups_week_{CURRENT_WEEK}.csv')

print(f"✓ Loaded {len(df_lineups)} lineup slots")
print(f"  Teams: {df_lineups['roster_id'].nunique()}")
print(f"  Total starters: {len(df_lineups)}")

# Show sample
print("\nSample lineup data:")
print(df_lineups.head(10)[['roster_id', 'team_name', 'slot', 'player_name', 'mu', 'sigma']].to_string(index=False))


✓ Loaded 108 lineup slots
  Teams: 12
  Total starters: 108

Sample lineup data:
 roster_id team_name slot            player_name        mu     sigma
         1    Team 1   QB          Waiver Pickup 16.795000 10.715864
         1    Team 1    K           Cairo Santos  8.466667  4.026578
         1    Team 1  DEF             NE Defense  6.050000  8.112953
         1    Team 1   RB         Bijan Robinson 20.515000  9.273094
         1    Team 1   RB Jacory Croskey-Merritt 10.320000  9.213295
         1    Team 1   WR           Emeka Egbuka 15.462500 10.170341
         1    Team 1   WR       Jameson Williams 11.580000 10.116758
         1    Team 1   TE           Trey McBride 15.362500  8.505439
         1    Team 1 FLEX             Tre Tucker 10.776667 10.055174
         2    Team 2   QB         Caleb Williams 19.420000  7.403675


## 🎲 Lognormal Parameterization Functions


In [3]:
def lognormal_params(mu: float, sigma: float) -> Tuple[float, float]:
    """
    Convert mean and std to lognormal parameters.
    
    Args:
        mu: Mean of the distribution
        sigma: Standard deviation of the distribution
        
    Returns:
        (mu_ln, sigma_ln): Lognormal distribution parameters
    """
    # Handle degenerate case (zero variance)
    if sigma == 0 or sigma < 1e-10:
        sigma_ln = 1e-9
        mu_ln = np.log(mu) if mu > 0 else 0
        return mu_ln, sigma_ln
    
    # Handle near-zero or negative mu
    if mu <= 0:
        mu = 1e-6
    
    # Calculate phi
    phi = np.sqrt(sigma**2 + mu**2)
    
    # Calculate lognormal parameters
    sigma_ln = np.sqrt(np.log((phi / mu)**2))
    mu_ln = np.log(mu**2 / phi)
    
    return mu_ln, sigma_ln


def simulate_player(mu: float, sigma: float, n_sims: int = N_SIMULATIONS) -> np.ndarray:
    """
    Simulate player points using lognormal distribution.
    
    Args:
        mu: Mean projected points
        sigma: Standard deviation of projected points
        n_sims: Number of simulations
        
    Returns:
        Array of simulated points (shape: n_sims)
    """
    mu_ln, sigma_ln = lognormal_params(mu, sigma)
    return np.random.lognormal(mu_ln, sigma_ln, size=n_sims)


print("✓ Lognormal parameterization functions defined")

# Test the functions
print("\nTest cases:")
test_cases = [
    (20.0, 5.0, "Normal case"),
    (15.0, 0.0, "Zero variance"),
    (10.0, 10.0, "High variance"),
    (0.0, 0.0, "Zero mean and variance")
]

for mu, sigma, desc in test_cases:
    mu_ln, sigma_ln = lognormal_params(mu, sigma)
    print(f"  {desc}: μ={mu:.1f}, σ={sigma:.1f} → μ_ln={mu_ln:.4f}, σ_ln={sigma_ln:.4f}")


✓ Lognormal parameterization functions defined

Test cases:
  Normal case: μ=20.0, σ=5.0 → μ_ln=2.9654, σ_ln=0.2462
  Zero variance: μ=15.0, σ=0.0 → μ_ln=2.7081, σ_ln=0.0000
  High variance: μ=10.0, σ=10.0 → μ_ln=1.9560, σ_ln=0.8326
  Zero mean and variance: μ=0.0, σ=0.0 → μ_ln=0.0000, σ_ln=0.0000


## 🎯 Run Monte Carlo Simulations


In [4]:
print("Running Monte Carlo simulations...\n")

# Reset seed for reproducibility
np.random.seed(SEED)

# Store all simulation results
all_simulations = []

# Get unique teams
teams = df_lineups['roster_id'].unique()

for team_id in teams:
    # Get this team's lineup
    team_lineup = df_lineups[df_lineups['roster_id'] == team_id].copy()
    team_name = team_lineup.iloc[0]['team_name']
    owner = team_lineup.iloc[0]['owner']
    
    n_players = len(team_lineup)
    
    print(f"Team {team_id}: {team_name} ({owner})")
    print(f"  Players in lineup: {n_players}")
    
    # Simulate each player
    player_simulations = []
    
    for idx, player in team_lineup.iterrows():
        mu = player['mu']
        sigma = player['sigma']
        
        # Simulate this player N_SIMULATIONS times
        player_sims = simulate_player(mu, sigma, N_SIMULATIONS)
        player_simulations.append(player_sims)
    
    # Stack into array (shape: n_sims x n_players)
    team_draws = np.column_stack(player_simulations)
    
    # Sum across players to get team totals (shape: n_sims)
    team_totals = team_draws.sum(axis=1)
    
    print(f"  Mean simulated total: {team_totals.mean():.2f}")
    print(f"  Median simulated total: {np.median(team_totals):.2f}")
    print(f"  Std simulated total: {team_totals.std():.2f}")
    print(f"  Min: {team_totals.min():.2f}, Max: {team_totals.max():.2f}")
    print()
    
    # Store results in long format
    for sim_id, total_points in enumerate(team_totals):
        all_simulations.append({
            'week': CURRENT_WEEK,
            'sim_id': sim_id,
            'team_id': team_id,
            'team_name': team_name,
            'owner': owner,
            'total_points': total_points
        })

# Create DataFrame
df_simulations = pd.DataFrame(all_simulations)

print(f"✓ Completed {N_SIMULATIONS:,} simulations for {len(teams)} teams")
print(f"✓ Total simulation records: {len(df_simulations):,}")


Running Monte Carlo simulations...

Team 1: Team 1 (xavierking4)
  Players in lineup: 9
  Mean simulated total: 115.31
  Median simulated total: 111.73
  Std simulated total: 27.47
  Min: 46.89, Max: 494.53

Team 2: Team 2 (asadrafique)
  Players in lineup: 9
  Mean simulated total: 110.39
  Median simulated total: 106.66
  Std simulated total: 26.89
  Min: 45.10, Max: 394.97

Team 3: Team 3 (amir812)
  Players in lineup: 9
  Mean simulated total: 124.96
  Median simulated total: 121.70
  Std simulated total: 27.35
  Min: 53.79, Max: 597.28

Team 4: Team 4 (umarrahman30)
  Players in lineup: 9
  Mean simulated total: 111.60
  Median simulated total: 108.24
  Std simulated total: 26.15
  Min: 42.22, Max: 434.11

Team 5: Team 5 (TBK41)
  Players in lineup: 9
  Mean simulated total: 116.20
  Median simulated total: 112.73
  Std simulated total: 26.51
  Min: 48.05, Max: 343.34

Team 6: Team 6 (Jibraan)
  Players in lineup: 9
  Mean simulated total: 127.13
  Median simulated total: 124.07
 

## 💾 Save Simulation Results


In [ ]:
# Save to CSV
output_file = f'../data/csv/monte_carlo_simulations_week_{CURRENT_WEEK}.csv'
df_simulations.to_csv(output_file, index=False)

print(f"✓ Saved simulation results to CSV: {output_file}")
print(f"  File size: {len(df_simulations):,} rows")

# Save to Database
conn_proj = sqlite3.connect('../data/databases/projections.db')
cursor = conn_proj.cursor()

# Create table for simulations
cursor.execute("""
    CREATE TABLE IF NOT EXISTS monte_carlo_simulations (
        run_id TEXT NOT NULL,
        week INTEGER NOT NULL,
        sim_id INTEGER NOT NULL,
        team_id INTEGER NOT NULL,
        team_name TEXT,
        owner TEXT,
        total_points REAL NOT NULL,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        PRIMARY KEY (run_id, week, sim_id, team_id)
    )
""")

# Create a run_id based on seed and timestamp
from datetime import datetime
run_id = f"seed_{SEED}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

# Add run_id to dataframe
df_simulations['run_id'] = run_id

# Insert simulations
df_simulations.to_sql('monte_carlo_simulations', conn_proj, if_exists='append', index=False)

conn_proj.commit()
conn_proj.close()

print(f"✓ Saved simulation results to database: projections.db")
print(f"  Run ID: {run_id}")
print(f"  Table: monte_carlo_simulations")


✓ Saved simulation results to CSV: monte_carlo_simulations_week_10.csv
  File size: 600,000 rows


NameError: name 'sqlite3' is not defined

### Save Simulation Run Metadata


In [ ]:
# Save metadata about this simulation run
conn_proj = sqlite3.connect('projections.db')
cursor = conn_proj.cursor()

# Create metadata table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS simulation_runs (
        run_id TEXT PRIMARY KEY,
        week INTEGER NOT NULL,
        seed INTEGER NOT NULL,
        n_simulations INTEGER NOT NULL,
        distribution_type TEXT,
        n_teams INTEGER,
        n_matchups INTEGER,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
""")

# Insert this run's metadata
cursor.execute("""
    INSERT OR REPLACE INTO simulation_runs 
    (run_id, week, seed, n_simulations, distribution_type, n_teams, n_matchups)
    VALUES (?, ?, ?, ?, ?, ?, ?)
""", (
    run_id,
    CURRENT_WEEK,
    SEED,
    N_SIMULATIONS,
    'lognormal',
    len(teams),
    len(matchups) if 'matchups' in locals() else 0
))

conn_proj.commit()
conn_proj.close()

print("✓ Saved simulation run metadata")
print(f"  Table: simulation_runs")


## 📊 Simulation Summary Statistics


In [ ]:
# Calculate summary statistics per team
summary_stats = df_simulations.groupby(['team_id', 'team_name', 'owner'])['total_points'].agg([
    ('mean', 'mean'),
    ('median', 'median'),
    ('std', 'std'),
    ('min', 'min'),
    ('max', 'max'),
    ('p10', lambda x: np.percentile(x, 10)),
    ('p25', lambda x: np.percentile(x, 25)),
    ('p75', lambda x: np.percentile(x, 75)),
    ('p90', lambda x: np.percentile(x, 90))
]).reset_index()

# Round for display
summary_stats = summary_stats.round(2)

# Sort by mean
summary_stats = summary_stats.sort_values('mean', ascending=False)

print("="*120)
print("MONTE CARLO SIMULATION SUMMARY")
print(f"Week {CURRENT_WEEK} | {N_SIMULATIONS:,} Simulations per Team | Seed: {SEED}")
print("="*120)
print()

display_cols = ['team_id', 'team_name', 'owner', 'mean', 'median', 'std', 'p10', 'p25', 'p75', 'p90']
print(summary_stats[display_cols].to_string(index=False))

print()
print("="*120)


## 📈 Visualization 1: Distribution of Team Outcomes


In [ ]:
# Create subplots for all teams
n_teams = len(teams)
n_cols = 3
n_rows = int(np.ceil(n_teams / n_cols))

fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 4*n_rows))
axes = axes.flatten()

# Sort teams by mean for consistent ordering
team_order = summary_stats.sort_values('mean', ascending=False)['team_id'].values

for idx, team_id in enumerate(team_order):
    ax = axes[idx]
    
    # Get data for this team
    team_data = df_simulations[df_simulations['team_id'] == team_id]
    team_name = team_data.iloc[0]['team_name']
    owner = team_data.iloc[0]['owner']
    points = team_data['total_points'].values
    
    # Plot histogram
    ax.hist(points, bins=50, alpha=0.7, color=f'C{idx}', edgecolor='black', linewidth=0.5)
    
    # Add mean and median lines
    mean_val = points.mean()
    median_val = np.median(points)
    
    ax.axvline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean_val:.1f}')
    ax.axvline(median_val, color='blue', linestyle=':', linewidth=2, label=f'Median: {median_val:.1f}')
    
    # Add percentiles
    p10 = np.percentile(points, 10)
    p90 = np.percentile(points, 90)
    ax.axvspan(p10, p90, alpha=0.2, color='gray', label=f'10-90%: {p10:.1f}-{p90:.1f}')
    
    ax.set_title(f'Team {team_id}: {team_name}\\n({owner})', fontsize=11, fontweight='bold')
    ax.set_xlabel('Total Points', fontsize=9)
    ax.set_ylabel('Frequency', fontsize=9)
    ax.legend(fontsize=8, loc='upper right')
    ax.grid(alpha=0.3)

# Hide unused subplots
for idx in range(len(team_order), len(axes)):
    axes[idx].axis('off')

plt.suptitle(f'Monte Carlo Simulations - Week {CURRENT_WEEK} Team Distributions\\n{N_SIMULATIONS:,} Simulations Each', 
             fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.savefig(f'../data/images/simulation_distributions_week_{CURRENT_WEEK}.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Saved: simulation_distributions_week_{CURRENT_WEEK}.png")


## 📊 Visualization 2: Box Plot Comparison


In [ ]:
# Create box plot
fig, ax = plt.subplots(figsize=(16, 8))

# Prepare data sorted by median
sorted_teams = summary_stats.sort_values('median', ascending=True)
team_labels = [f"{row['team_name']}\\n({row['owner']})" for _, row in sorted_teams.iterrows()]

# Get data in order
data_to_plot = [df_simulations[df_simulations['team_id'] == tid]['total_points'].values 
                for tid in sorted_teams['team_id']]

# Create box plot
bp = ax.boxplot(data_to_plot, 
                labels=team_labels,
                patch_artist=True,
                showmeans=True,
                meanprops=dict(marker='D', markerfacecolor='red', markersize=8),
                medianprops=dict(color='darkblue', linewidth=2),
                whiskerprops=dict(linewidth=1.5),
                capprops=dict(linewidth=1.5),
                flierprops=dict(marker='o', markerfacecolor='gray', markersize=3, alpha=0.3))

# Color boxes
colors = sns.color_palette('husl', len(team_labels))
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax.set_xlabel('Team', fontsize=12, fontweight='bold')
ax.set_ylabel('Total Points', fontsize=12, fontweight='bold')
ax.set_title(f'Monte Carlo Simulations - Week {CURRENT_WEEK} Box Plot Comparison\\n{N_SIMULATIONS:,} Simulations per Team', 
             fontsize=14, fontweight='bold', pad=20)
ax.grid(axis='y', alpha=0.3)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig(f'../data/images/simulation_boxplot_week_{CURRENT_WEEK}.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Saved: simulation_boxplot_week_{CURRENT_WEEK}.png")


## 📊 Visualization 3: Violin Plot with Percentiles


In [ ]:
# Create violin plot
fig, ax = plt.subplots(figsize=(16, 8))

# Prepare data
plot_df = df_simulations.merge(summary_stats[['team_id', 'median']], on='team_id')
plot_df = plot_df.sort_values('median')
plot_df['team_label'] = plot_df['team_name'] + '\\n(' + plot_df['owner'] + ')'

# Create violin plot
sns.violinplot(data=plot_df, x='team_label', y='total_points', 
               palette='husl', inner='quartile', ax=ax)

# Add mean points
for i, (team_label, group) in enumerate(plot_df.groupby('team_label', sort=False)):
    mean_val = group['total_points'].mean()
    ax.plot(i, mean_val, 'r*', markersize=15, markeredgecolor='black', markeredgewidth=0.5)

ax.set_xlabel('Team', fontsize=12, fontweight='bold')
ax.set_ylabel('Total Points', fontsize=12, fontweight='bold')
ax.set_title(f'Monte Carlo Simulations - Week {CURRENT_WEEK} Violin Plot\\n{N_SIMULATIONS:,} Simulations per Team (Red ★ = Mean)', 
             fontsize=14, fontweight='bold', pad=20)
ax.grid(axis='y', alpha=0.3)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig(f'../data/images/simulation_violin_week_{CURRENT_WEEK}.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Saved: simulation_violin_week_{CURRENT_WEEK}.png")


## 📊 Visualization 4: Win Probability Heatmap


In [ ]:
# Calculate head-to-head win probabilities
print("Calculating head-to-head win probabilities...\n")

teams_list = sorted(df_simulations['team_id'].unique())
n_teams_list = len(teams_list)

# Create win probability matrix
win_prob_matrix = np.zeros((n_teams_list, n_teams_list))

for i, team_a in enumerate(teams_list):
    points_a = df_simulations[df_simulations['team_id'] == team_a]['total_points'].values
    
    for j, team_b in enumerate(teams_list):
        if i == j:
            win_prob_matrix[i, j] = 0.5  # Tie probability
        else:
            points_b = df_simulations[df_simulations['team_id'] == team_b]['total_points'].values
            wins = np.sum(points_a > points_b)
            win_prob_matrix[i, j] = wins / N_SIMULATIONS

# Create heatmap
fig, ax = plt.subplots(figsize=(14, 12))

# Get team names for labels
team_names = [df_simulations[df_simulations['team_id'] == tid]['team_name'].iloc[0] for tid in teams_list]

# Plot heatmap
im = ax.imshow(win_prob_matrix, cmap='RdYlGn', aspect='auto', vmin=0, vmax=1)

# Add colorbar
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Win Probability', rotation=270, labelpad=20, fontsize=12, fontweight='bold')

# Set ticks and labels
ax.set_xticks(np.arange(n_teams_list))
ax.set_yticks(np.arange(n_teams_list))
ax.set_xticklabels(team_names, rotation=45, ha='right')
ax.set_yticklabels(team_names)

# Add text annotations
for i in range(n_teams_list):
    for j in range(n_teams_list):
        if i != j:
            text = ax.text(j, i, f'{win_prob_matrix[i, j]:.3f}',
                          ha="center", va="center", color="black", fontsize=8)

ax.set_xlabel('Opponent', fontsize=12, fontweight='bold')
ax.set_ylabel('Team', fontsize=12, fontweight='bold')
ax.set_title(f'Head-to-Head Win Probability Matrix - Week {CURRENT_WEEK}\\n{N_SIMULATIONS:,} Monte Carlo Simulations', 
             fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
plt.savefig(f'../data/images/win_probability_heatmap_week_{CURRENT_WEEK}.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Saved: win_probability_heatmap_week_{CURRENT_WEEK}.png")


## 📊 Visualization 5: Percentile Ranges Comparison


In [ ]:
# Create percentile range plot
fig, ax = plt.subplots(figsize=(14, 10))

# Sort teams by median
sorted_teams = summary_stats.sort_values('median', ascending=True)

y_positions = np.arange(len(sorted_teams))

for idx, (_, row) in enumerate(sorted_teams.iterrows()):
    team_id = row['team_id']
    team_name = row['team_name']
    owner = row['owner']
    
    # Get percentiles
    p10 = row['p10']
    p25 = row['p25']
    median = row['median']
    p75 = row['p75']
    p90 = row['p90']
    mean = row['mean']
    
    # Plot ranges
    # 10-90 percentile (thin line)
    ax.plot([p10, p90], [idx, idx], 'k-', linewidth=2, alpha=0.3)
    
    # 25-75 percentile (thick line)
    ax.plot([p25, p75], [idx, idx], color=f'C{idx}', linewidth=8, alpha=0.7)
    
    # Median (circle)
    ax.plot(median, idx, 'o', color='blue', markersize=10, markeredgecolor='darkblue', markeredgewidth=2)
    
    # Mean (diamond)
    ax.plot(mean, idx, 'D', color='red', markersize=8, markeredgecolor='darkred', markeredgewidth=1.5)

# Set labels
team_labels = [f"{row['team_name']} ({row['owner']})" for _, row in sorted_teams.iterrows()]
ax.set_yticks(y_positions)
ax.set_yticklabels(team_labels)
ax.set_xlabel('Total Points', fontsize=12, fontweight='bold')
ax.set_ylabel('Team', fontsize=12, fontweight='bold')
ax.set_title(f'Monte Carlo Simulations - Week {CURRENT_WEEK} Percentile Ranges\\n{N_SIMULATIONS:,} Simulations per Team', 
             fontsize=14, fontweight='bold', pad=20)

# Add legend
from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], color='gray', linewidth=8, alpha=0.7, label='25th-75th Percentile'),
    Line2D([0], [0], color='black', linewidth=2, alpha=0.3, label='10th-90th Percentile'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markersize=10, 
           markeredgecolor='darkblue', markeredgewidth=2, label='Median'),
    Line2D([0], [0], marker='D', color='w', markerfacecolor='red', markersize=8, 
           markeredgecolor='darkred', markeredgewidth=1.5, label='Mean')
]
ax.legend(handles=legend_elements, loc='lower right', fontsize=10)

ax.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.savefig(f'../data/images/percentile_ranges_week_{CURRENT_WEEK}.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Saved: percentile_ranges_week_{CURRENT_WEEK}.png")


## 📊 Visualization 6: Cumulative Distribution Functions


In [ ]:
# Create CDF plot for all teams
fig, ax = plt.subplots(figsize=(14, 8))

# Sort teams by mean
sorted_teams = summary_stats.sort_values('mean', ascending=False)

for idx, (_, row) in enumerate(sorted_teams.iterrows()):
    team_id = row['team_id']
    team_name = row['team_name']
    owner = row['owner']
    
    # Get points
    points = df_simulations[df_simulations['team_id'] == team_id]['total_points'].values
    
    # Sort for CDF
    sorted_points = np.sort(points)
    cdf = np.arange(1, len(sorted_points) + 1) / len(sorted_points)
    
    # Plot
    ax.plot(sorted_points, cdf, linewidth=2, label=f'{team_name}', alpha=0.8)

ax.set_xlabel('Total Points', fontsize=12, fontweight='bold')
ax.set_ylabel('Cumulative Probability', fontsize=12, fontweight='bold')
ax.set_title(f'Monte Carlo Simulations - Week {CURRENT_WEEK} Cumulative Distribution Functions\\n{N_SIMULATIONS:,} Simulations per Team', 
             fontsize=14, fontweight='bold', pad=20)
ax.grid(alpha=0.3)
ax.legend(loc='lower right', fontsize=9, ncol=2)

# Add reference lines
ax.axhline(0.5, color='gray', linestyle='--', alpha=0.5, linewidth=1)
ax.axhline(0.1, color='gray', linestyle=':', alpha=0.3, linewidth=1)
ax.axhline(0.9, color='gray', linestyle=':', alpha=0.3, linewidth=1)

plt.tight_layout()
plt.savefig(f'../data/images/cdf_plot_week_{CURRENT_WEEK}.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Saved: cdf_plot_week_{CURRENT_WEEK}.png")


## 🏆 Final Summary Report


In [ ]:
print("="*100)
print("MONTE CARLO SIMULATION - FINAL REPORT")
print("="*100)
print()
print(f"Week: {CURRENT_WEEK}")
print(f"Simulations per Team: {N_SIMULATIONS:,}")
print(f"Random Seed: {SEED}")
print(f"Distribution: Lognormal")
print()
print("Top 5 Teams by Mean Projected Points:")
print("-" * 100)
top5 = summary_stats.head(5)
for idx, (_, row) in enumerate(top5.iterrows(), 1):
    print(f"  {idx}. {row['team_name']} ({row['owner']})")
    print(f"     Mean: {row['mean']:.2f} | Median: {row['median']:.2f} | Std: {row['std']:.2f}")
    print(f"     Range: {row['min']:.2f} - {row['max']:.2f} | 80% CI: [{row['p10']:.2f}, {row['p90']:.2f}]")
    print()

print("Files Generated:")
print("-" * 100)
print(f"  • monte_carlo_simulations_week_{CURRENT_WEEK}.csv")
print(f"  • simulation_distributions_week_{CURRENT_WEEK}.png")
print(f"  • simulation_boxplot_week_{CURRENT_WEEK}.png")
print(f"  • simulation_violin_week_{CURRENT_WEEK}.png")
print(f"  • win_probability_heatmap_week_{CURRENT_WEEK}.png")
print(f"  • percentile_ranges_week_{CURRENT_WEEK}.png")
print(f"  • cdf_plot_week_{CURRENT_WEEK}.png")
print()
print("="*100)
print("✓ Analysis Complete!")
print("="*100)


## 🎰 Betting Odds Analysis

Generate betting odds based on Monte Carlo simulations (American odds, no vig)


In [ ]:
import sqlite3

# Load matchups for the current week
conn_league = sqlite3.connect('../data/databases/league.db')
query_matchups = f"""
    SELECT 
        roster_id,
        matchup_id_number
    FROM matchups
    WHERE week = {CURRENT_WEEK}
    ORDER BY matchup_id_number, roster_id
"""
df_matchups = pd.read_sql_query(query_matchups, conn_league)
conn_league.close()

# Group into pairs
matchups = df_matchups.groupby('matchup_id_number')['roster_id'].apply(list).tolist()

print(f"✓ Loaded {len(matchups)} matchups for Week {CURRENT_WEEK}\n")
print("Matchups:")
for idx, matchup in enumerate(matchups, 1):
    if len(matchup) == 2:
        team1_name = df_simulations[df_simulations['team_id'] == matchup[0]]['team_name'].iloc[0]
        team2_name = df_simulations[df_simulations['team_id'] == matchup[1]]['team_name'].iloc[0]
        print(f"  {idx}. Team {matchup[0]} ({team1_name}) vs Team {matchup[1]} ({team2_name})")
    else:
        print(f"  {idx}. Incomplete matchup: {matchup}")


### Odds Conversion Functions


In [ ]:
def probability_to_american_odds(prob: float) -> str:
    """
    Convert probability to American odds (no vig).
    
    Args:
        prob: Probability between 0 and 1
        
    Returns:
        American odds as string (e.g., "+150" or "-200")
    """
    if prob >= 1.0:
        return "-∞"
    if prob <= 0.0:
        return "+∞"
    
    if prob >= 0.5:
        # Favorite (negative odds)
        odds = -(prob / (1 - prob)) * 100
        return f"{int(round(odds))}"
    else:
        # Underdog (positive odds)
        odds = ((1 - prob) / prob) * 100
        return f"+{int(round(odds))}"


def format_odds_display(prob: float, odds: str) -> str:
    """Format odds with probability for display."""
    return f"{odds:>6} ({prob*100:5.2f}%)"


print("✓ Odds conversion functions defined")

# Test
test_probs = [0.75, 0.60, 0.50, 0.40, 0.25, 0.10]
print("\nTest probabilities → American odds:")
for p in test_probs:
    odds = probability_to_american_odds(p)
    print(f"  {p*100:5.1f}% → {odds}")


### 1. Team Over/Under Odds (Based on Median)


In [ ]:
print("="*100)
print("1. TEAM OVER/UNDER ODDS")
print("="*100)
print("\nLine set at each team's median. Odds based on simulation results.\n")

team_ou_odds = []

for _, team_row in summary_stats.iterrows():
    team_id = team_row['team_id']
    team_name = team_row['team_name']
    owner = team_row['owner']
    median = team_row['median']
    
    # Get simulation results for this team
    team_sims = df_simulations[df_simulations['team_id'] == team_id]['total_points'].values
    
    # Calculate probabilities
    over_count = np.sum(team_sims > median)
    under_count = np.sum(team_sims < median)
    push_count = np.sum(team_sims == median)
    
    over_prob = over_count / N_SIMULATIONS
    under_prob = under_count / N_SIMULATIONS
    
    # Convert to odds
    over_odds = probability_to_american_odds(over_prob)
    under_odds = probability_to_american_odds(under_prob)
    
    team_ou_odds.append({
        'team_id': team_id,
        'team_name': team_name,
        'owner': owner,
        'line': median,
        'over_prob': over_prob,
        'over_odds': over_odds,
        'under_prob': under_prob,
        'under_odds': under_odds,
        'push_count': push_count
    })
    
    print(f"Team {team_id}: {team_name} ({owner})")
    print(f"  O/U Line: {median:.1f}")
    print(f"  Over  {format_odds_display(over_prob, over_odds)}")
    print(f"  Under {format_odds_display(under_prob, under_odds)}")
    if push_count > 0:
        print(f"  Pushes: {push_count}/{N_SIMULATIONS}")
    print()

df_team_ou = pd.DataFrame(team_ou_odds)
print("="*100)


### 2. Matchup Total Over/Under Odds


In [ ]:
print("="*100)
print("2. MATCHUP TOTAL OVER/UNDER ODDS")
print("="*100)
print("\nCombined total for each matchup. Line set at median of combined totals.\n")

matchup_ou_odds = []

for idx, matchup in enumerate(matchups, 1):
    if len(matchup) != 2:
        continue
    
    team1_id, team2_id = matchup
    team1_name = df_simulations[df_simulations['team_id'] == team1_id]['team_name'].iloc[0]
    team2_name = df_simulations[df_simulations['team_id'] == team2_id]['team_name'].iloc[0]
    
    # Get simulation results for both teams
    team1_sims = df_simulations[df_simulations['team_id'] == team1_id].sort_values('sim_id')['total_points'].values
    team2_sims = df_simulations[df_simulations['team_id'] == team2_id].sort_values('sim_id')['total_points'].values
    
    # Calculate combined totals for each simulation
    combined_totals = team1_sims + team2_sims
    
    # Set line at median
    line = np.median(combined_totals)
    
    # Calculate probabilities
    over_count = np.sum(combined_totals > line)
    under_count = np.sum(combined_totals < line)
    
    over_prob = over_count / N_SIMULATIONS
    under_prob = under_count / N_SIMULATIONS
    
    # Convert to odds
    over_odds = probability_to_american_odds(over_prob)
    under_odds = probability_to_american_odds(under_prob)
    
    matchup_ou_odds.append({
        'matchup': f"Team {team1_id} vs Team {team2_id}",
        'team1_id': team1_id,
        'team1_name': team1_name,
        'team2_id': team2_id,
        'team2_name': team2_name,
        'line': line,
        'over_prob': over_prob,
        'over_odds': over_odds,
        'under_prob': under_prob,
        'under_odds': under_odds
    })
    
    print(f"Matchup {idx}: {team1_name} vs {team2_name}")
    print(f"  O/U Line: {line:.1f}")
    print(f"  Over  {format_odds_display(over_prob, over_odds)}")
    print(f"  Under {format_odds_display(under_prob, under_odds)}")
    print()

df_matchup_ou = pd.DataFrame(matchup_ou_odds)
print("="*100)


### 3. Matchup Moneyline Odds


In [ ]:
print("="*100)
print("3. MATCHUP MONEYLINE ODDS")
print("="*100)
print("\nWin probabilities based on head-to-head simulation results.\n")

matchup_ml_odds = []

for idx, matchup in enumerate(matchups, 1):
    if len(matchup) != 2:
        continue
    
    team1_id, team2_id = matchup
    team1_name = df_simulations[df_simulations['team_id'] == team1_id]['team_name'].iloc[0]
    team2_name = df_simulations[df_simulations['team_id'] == team2_id]['team_name'].iloc[0]
    
    # Get simulation results for both teams
    team1_sims = df_simulations[df_simulations['team_id'] == team1_id].sort_values('sim_id')['total_points'].values
    team2_sims = df_simulations[df_simulations['team_id'] == team2_id].sort_values('sim_id')['total_points'].values
    
    # Calculate win probabilities
    team1_wins = np.sum(team1_sims > team2_sims)
    team2_wins = np.sum(team2_sims > team1_sims)
    ties = np.sum(team1_sims == team2_sims)
    
    team1_win_prob = team1_wins / N_SIMULATIONS
    team2_win_prob = team2_wins / N_SIMULATIONS
    
    # Convert to moneyline odds
    team1_ml = probability_to_american_odds(team1_win_prob)
    team2_ml = probability_to_american_odds(team2_win_prob)
    
    matchup_ml_odds.append({
        'matchup': f"Team {team1_id} vs Team {team2_id}",
        'team1_id': team1_id,
        'team1_name': team1_name,
        'team1_win_prob': team1_win_prob,
        'team1_ml': team1_ml,
        'team2_id': team2_id,
        'team2_name': team2_name,
        'team2_win_prob': team2_win_prob,
        'team2_ml': team2_ml,
        'ties': ties
    })
    
    print(f"Matchup {idx}: {team1_name} vs {team2_name}")
    print(f"  {team1_name:25} {format_odds_display(team1_win_prob, team1_ml)}")
    print(f"  {team2_name:25} {format_odds_display(team2_win_prob, team2_ml)}")
    if ties > 0:
        print(f"  Ties: {ties}/{N_SIMULATIONS}")
    print()

df_matchup_ml = pd.DataFrame(matchup_ml_odds)
print("="*100)


### 4. Highest Scoring Team Odds


In [ ]:
print("="*100)
print("4. HIGHEST SCORING TEAM OF THE WEEK ODDS")
print("="*100)
print("\nProbability of each team being the highest scorer in a given week.\n")

highest_scorer_odds = []

# For each simulation, find the highest scoring team
high_score_winners = []

for sim_id in range(N_SIMULATIONS):
    sim_data = df_simulations[df_simulations['sim_id'] == sim_id]
    max_score = sim_data['total_points'].max()
    # Handle ties - all teams with max score
    winners = sim_data[sim_data['total_points'] == max_score]['team_id'].tolist()
    high_score_winners.append(winners)

# Count how many times each team was the highest scorer
team_high_score_counts = {}
for team_id in df_simulations['team_id'].unique():
    count = sum(1 for winners in high_score_winners if team_id in winners)
    team_high_score_counts[team_id] = count

# Calculate probabilities and odds
for team_id in sorted(team_high_score_counts.keys()):
    count = team_high_score_counts[team_id]
    prob = count / N_SIMULATIONS
    odds = probability_to_american_odds(prob)
    
    team_name = df_simulations[df_simulations['team_id'] == team_id]['team_name'].iloc[0]
    owner = df_simulations[df_simulations['team_id'] == team_id]['owner'].iloc[0]
    
    highest_scorer_odds.append({
        'team_id': team_id,
        'team_name': team_name,
        'owner': owner,
        'count': count,
        'probability': prob,
        'odds': odds
    })

# Sort by probability descending
highest_scorer_odds.sort(key=lambda x: x['probability'], reverse=True)

print(f"{'Rank':<5} {'Team':<25} {'Owner':<18} {'Odds':>8} {'Probability':>12} {'Count':>8}")
print("-" * 100)

for rank, team in enumerate(highest_scorer_odds, 1):
    print(f"{rank:<5} {team['team_name']:<25} {team['owner']:<18} {team['odds']:>8} {team['probability']*100:>11.2f}% {team['count']:>8}")

df_highest_scorer = pd.DataFrame(highest_scorer_odds)
print("\n" + "="*100)


### 5. Lowest Scoring Team Odds


In [ ]:
print("="*100)
print("5. LOWEST SCORING TEAM OF THE WEEK ODDS")
print("="*100)
print("\nProbability of each team being the lowest scorer in a given week.\n")

lowest_scorer_odds = []

# For each simulation, find the lowest scoring team
low_score_losers = []

for sim_id in range(N_SIMULATIONS):
    sim_data = df_simulations[df_simulations['sim_id'] == sim_id]
    min_score = sim_data['total_points'].min()
    # Handle ties - all teams with min score
    losers = sim_data[sim_data['total_points'] == min_score]['team_id'].tolist()
    low_score_losers.append(losers)

# Count how many times each team was the lowest scorer
team_low_score_counts = {}
for team_id in df_simulations['team_id'].unique():
    count = sum(1 for losers in low_score_losers if team_id in losers)
    team_low_score_counts[team_id] = count

# Calculate probabilities and odds
for team_id in sorted(team_low_score_counts.keys()):
    count = team_low_score_counts[team_id]
    prob = count / N_SIMULATIONS
    odds = probability_to_american_odds(prob)
    
    team_name = df_simulations[df_simulations['team_id'] == team_id]['team_name'].iloc[0]
    owner = df_simulations[df_simulations['team_id'] == team_id]['owner'].iloc[0]
    
    lowest_scorer_odds.append({
        'team_id': team_id,
        'team_name': team_name,
        'owner': owner,
        'count': count,
        'probability': prob,
        'odds': odds
    })

# Sort by probability descending
lowest_scorer_odds.sort(key=lambda x: x['probability'], reverse=True)

print(f"{'Rank':<5} {'Team':<25} {'Owner':<18} {'Odds':>8} {'Probability':>12} {'Count':>8}")
print("-" * 100)

for rank, team in enumerate(lowest_scorer_odds, 1):
    print(f"{rank:<5} {team['team_name']:<25} {team['owner']:<18} {team['odds']:>8} {team['probability']*100:>11.2f}% {team['count']:>8}")

df_lowest_scorer = pd.DataFrame(lowest_scorer_odds)
print("\n" + "="*100)


### 💾 Save Betting Odds to CSV


In [ ]:
# Save all betting odds to CSV files
df_team_ou.to_csv(f'../data/csv/betting_odds_team_ou_week_{CURRENT_WEEK}.csv', index=False)
df_matchup_ou.to_csv(f'../data/csv/betting_odds_matchup_ou_week_{CURRENT_WEEK}.csv', index=False)
df_matchup_ml.to_csv(f'../data/csv/betting_odds_matchup_ml_week_{CURRENT_WEEK}.csv', index=False)
df_highest_scorer.to_csv(f'../data/csv/betting_odds_highest_scorer_week_{CURRENT_WEEK}.csv', index=False)
df_lowest_scorer.to_csv(f'../data/csv/betting_odds_lowest_scorer_week_{CURRENT_WEEK}.csv', index=False)

print("✓ Saved betting odds to CSV files:")
print(f"  • betting_odds_team_ou_week_{CURRENT_WEEK}.csv")
print(f"  • betting_odds_matchup_ou_week_{CURRENT_WEEK}.csv")
print(f"  • betting_odds_matchup_ml_week_{CURRENT_WEEK}.csv")
print(f"  • betting_odds_highest_scorer_week_{CURRENT_WEEK}.csv")
print(f"  • betting_odds_lowest_scorer_week_{CURRENT_WEEK}.csv")

# Save all betting odds to Database
conn_proj = sqlite3.connect('../data/databases/projections.db')
cursor = conn_proj.cursor()

# Create tables for betting odds
cursor.execute("""
    CREATE TABLE IF NOT EXISTS betting_odds_team_ou (
        run_id TEXT NOT NULL,
        week INTEGER NOT NULL,
        team_id INTEGER NOT NULL,
        team_name TEXT,
        owner TEXT,
        line REAL,
        over_prob REAL,
        over_odds TEXT,
        under_prob REAL,
        under_odds TEXT,
        push_count INTEGER,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        PRIMARY KEY (run_id, week, team_id)
    )
""")

cursor.execute("""
    CREATE TABLE IF NOT EXISTS betting_odds_matchup_ou (
        run_id TEXT NOT NULL,
        week INTEGER NOT NULL,
        matchup TEXT,
        team1_id INTEGER,
        team1_name TEXT,
        team2_id INTEGER,
        team2_name TEXT,
        line REAL,
        over_prob REAL,
        over_odds TEXT,
        under_prob REAL,
        under_odds TEXT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        PRIMARY KEY (run_id, week, team1_id, team2_id)
    )
""")

cursor.execute("""
    CREATE TABLE IF NOT EXISTS betting_odds_matchup_ml (
        run_id TEXT NOT NULL,
        week INTEGER NOT NULL,
        matchup TEXT,
        team1_id INTEGER,
        team1_name TEXT,
        team1_win_prob REAL,
        team1_ml TEXT,
        team2_id INTEGER,
        team2_name TEXT,
        team2_win_prob REAL,
        team2_ml TEXT,
        ties INTEGER,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        PRIMARY KEY (run_id, week, team1_id, team2_id)
    )
""")

cursor.execute("""
    CREATE TABLE IF NOT EXISTS betting_odds_highest_scorer (
        run_id TEXT NOT NULL,
        week INTEGER NOT NULL,
        team_id INTEGER NOT NULL,
        team_name TEXT,
        owner TEXT,
        count INTEGER,
        probability REAL,
        odds TEXT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        PRIMARY KEY (run_id, week, team_id)
    )
""")

cursor.execute("""
    CREATE TABLE IF NOT EXISTS betting_odds_lowest_scorer (
        run_id TEXT NOT NULL,
        week INTEGER NOT NULL,
        team_id INTEGER NOT NULL,
        team_name TEXT,
        owner TEXT,
        count INTEGER,
        probability REAL,
        odds TEXT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        PRIMARY KEY (run_id, week, team_id)
    )
""")

# Add metadata
df_team_ou['run_id'] = run_id
df_team_ou['week'] = CURRENT_WEEK
df_matchup_ou['run_id'] = run_id
df_matchup_ou['week'] = CURRENT_WEEK
df_matchup_ml['run_id'] = run_id
df_matchup_ml['week'] = CURRENT_WEEK
df_highest_scorer['run_id'] = run_id
df_highest_scorer['week'] = CURRENT_WEEK
df_lowest_scorer['run_id'] = run_id
df_lowest_scorer['week'] = CURRENT_WEEK

# Insert data
df_team_ou.to_sql('betting_odds_team_ou', conn_proj, if_exists='append', index=False)
df_matchup_ou.to_sql('betting_odds_matchup_ou', conn_proj, if_exists='append', index=False)
df_matchup_ml.to_sql('betting_odds_matchup_ml', conn_proj, if_exists='append', index=False)
df_highest_scorer.to_sql('betting_odds_highest_scorer', conn_proj, if_exists='append', index=False)
df_lowest_scorer.to_sql('betting_odds_lowest_scorer', conn_proj, if_exists='append', index=False)

conn_proj.commit()
conn_proj.close()

print("\n✓ Saved betting odds to database: projections.db")
print(f"  Run ID: {run_id}")
print("  Tables:")
print("    • betting_odds_team_ou")
print("    • betting_odds_matchup_ou")
print("    • betting_odds_matchup_ml")
print("    • betting_odds_highest_scorer")
print("    • betting_odds_lowest_scorer")


## 📊 Betting Odds Visualizations


In [ ]:
# Create visualization for highest/lowest scorer odds
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))

# Highest scorer
high_data = df_highest_scorer.sort_values('probability', ascending=True)
y_pos = np.arange(len(high_data))
colors_high = plt.cm.Greens(np.linspace(0.3, 0.9, len(high_data)))

ax1.barh(y_pos, high_data['probability']*100, color=colors_high, edgecolor='black', linewidth=0.5)
ax1.set_yticks(y_pos)
ax1.set_yticklabels(high_data['team_name'])
ax1.set_xlabel('Probability (%)', fontsize=12, fontweight='bold')
ax1.set_title(f'Highest Scoring Team Odds - Week {CURRENT_WEEK}', fontsize=14, fontweight='bold')
ax1.grid(axis='x', alpha=0.3)

# Add odds labels
for i, (prob, odds) in enumerate(zip(high_data['probability'], high_data['odds'])):
    ax1.text(prob*100 + 0.5, i, f"{odds}", va='center', fontsize=9, fontweight='bold')

# Lowest scorer
low_data = df_lowest_scorer.sort_values('probability', ascending=True)
y_pos = np.arange(len(low_data))
colors_low = plt.cm.Reds(np.linspace(0.3, 0.9, len(low_data)))

ax2.barh(y_pos, low_data['probability']*100, color=colors_low, edgecolor='black', linewidth=0.5)
ax2.set_yticks(y_pos)
ax2.set_yticklabels(low_data['team_name'])
ax2.set_xlabel('Probability (%)', fontsize=12, fontweight='bold')
ax2.set_title(f'Lowest Scoring Team Odds - Week {CURRENT_WEEK}', fontsize=14, fontweight='bold')
ax2.grid(axis='x', alpha=0.3)

# Add odds labels
for i, (prob, odds) in enumerate(zip(low_data['probability'], low_data['odds'])):
    ax2.text(prob*100 + 0.5, i, f"{odds}", va='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.savefig(f'../data/images/betting_odds_high_low_week_{CURRENT_WEEK}.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Saved: betting_odds_high_low_week_{CURRENT_WEEK}.png")


In [ ]:
# Create matchup moneyline visualization
if len(df_matchup_ml) > 0:
    fig, ax = plt.subplots(figsize=(14, len(df_matchup_ml) * 1.5))
    
    y_pos = []
    labels = []
    probabilities = []
    colors = []
    odds_labels = []
    
    for idx, row in df_matchup_ml.iterrows():
        # Team 1
        y_pos.append(idx * 2)
        labels.append(f"{row['team1_name']}")
        probabilities.append(row['team1_win_prob'] * 100)
        colors.append('steelblue' if row['team1_win_prob'] > 0.5 else 'lightcoral')
        odds_labels.append(row['team1_ml'])
        
        # Team 2
        y_pos.append(idx * 2 + 0.8)
        labels.append(f"{row['team2_name']}")
        probabilities.append(row['team2_win_prob'] * 100)
        colors.append('steelblue' if row['team2_win_prob'] > 0.5 else 'lightcoral')
        odds_labels.append(row['team2_ml'])
    
    ax.barh(y_pos, probabilities, color=colors, edgecolor='black', linewidth=1, height=0.7)
    ax.set_yticks(y_pos)
    ax.set_yticklabels(labels)
    ax.set_xlabel('Win Probability (%)', fontsize=12, fontweight='bold')
    ax.set_title(f'Matchup Moneyline Odds - Week {CURRENT_WEEK}', fontsize=14, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)
    
    # Add odds labels
    for y, prob, odds in zip(y_pos, probabilities, odds_labels):
        ax.text(prob + 1, y, f"{odds}", va='center', fontsize=10, fontweight='bold')
    
    # Add matchup separators
    for i in range(1, len(df_matchup_ml)):
        ax.axhline(y=i*2 - 0.5, color='gray', linestyle='--', alpha=0.5, linewidth=2)
    
    plt.tight_layout()
    plt.savefig(f'../data/images/betting_odds_moneyline_week_{CURRENT_WEEK}.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"✓ Saved: betting_odds_moneyline_week_{CURRENT_WEEK}.png")
else:
    print("⚠ No matchups to visualize")


## 📋 Updated Final Summary


In [ ]:
print("="*100)
print("MONTE CARLO SIMULATION & BETTING ODDS - FINAL REPORT")
print("="*100)
print()
print(f"Week: {CURRENT_WEEK}")
print(f"Simulations per Team: {N_SIMULATIONS:,}")
print(f"Random Seed: {SEED}")
print(f"Run ID: {run_id}")
print(f"Distribution: Lognormal")
print(f"Odds Format: American (No Vig)")
print()

print("Betting Markets Generated:")
print("-" * 100)
print(f"  1. Team Over/Under ({len(df_team_ou)} teams)")
print(f"  2. Matchup Total O/U ({len(df_matchup_ou)} matchups)")
print(f"  3. Matchup Moneyline ({len(df_matchup_ml)} matchups)")
print(f"  4. Highest Scorer Props ({len(df_highest_scorer)} teams)")
print(f"  5. Lowest Scorer Props ({len(df_lowest_scorer)} teams)")
print()

print("Top Favorites (Highest Scorer):")
print("-" * 100)
for rank, team in enumerate(df_highest_scorer.head(3).itertuples(), 1):
    print(f"  {rank}. {team.team_name:25} {team.odds:>8} ({team.probability*100:5.2f}%)")
print()

print("Top Underdogs (Lowest Scorer):")
print("-" * 100)
for rank, team in enumerate(df_lowest_scorer.head(3).itertuples(), 1):
    print(f"  {rank}. {team.team_name:25} {team.odds:>8} ({team.probability*100:5.2f}%)")
print()

print("Database Storage (projections.db):")
print("-" * 100)
print("  Simulation Tables:")
print("    • simulation_runs (metadata)")
print("    • monte_carlo_simulations (120,000 rows)")
print()
print("  Betting Odds Tables:")
print("    • betting_odds_team_ou")
print("    • betting_odds_matchup_ou")
print("    • betting_odds_matchup_ml")
print("    • betting_odds_highest_scorer")
print("    • betting_odds_lowest_scorer")
print()

print("CSV Files Generated:")
print("-" * 100)
print("  Simulation Data:")
print(f"    • monte_carlo_simulations_week_{CURRENT_WEEK}.csv")
print()
print("  Betting Odds CSV:")
print(f"    • betting_odds_team_ou_week_{CURRENT_WEEK}.csv")
print(f"    • betting_odds_matchup_ou_week_{CURRENT_WEEK}.csv")
print(f"    • betting_odds_matchup_ml_week_{CURRENT_WEEK}.csv")
print(f"    • betting_odds_highest_scorer_week_{CURRENT_WEEK}.csv")
print(f"    • betting_odds_lowest_scorer_week_{CURRENT_WEEK}.csv")
print()

print("Visualizations:")
print("-" * 100)
print(f"  • simulation_distributions_week_{CURRENT_WEEK}.png")
print(f"  • simulation_boxplot_week_{CURRENT_WEEK}.png")
print(f"  • simulation_violin_week_{CURRENT_WEEK}.png")
print(f"  • win_probability_heatmap_week_{CURRENT_WEEK}.png")
print(f"  • percentile_ranges_week_{CURRENT_WEEK}.png")
print(f"  • cdf_plot_week_{CURRENT_WEEK}.png")
print(f"  • betting_odds_high_low_week_{CURRENT_WEEK}.png")
print(f"  • betting_odds_moneyline_week_{CURRENT_WEEK}.png")
print()
print("="*100)
print("✓ Complete Analysis with Betting Odds!")
print("✓ All data stored in projections.db")
print("="*100)


## 💾 Database Query Examples

How to retrieve simulation and betting odds data from the database


In [ ]:
# Example queries to retrieve data from the database

conn_proj = sqlite3.connect('projections.db')

print("="*100)
print("DATABASE QUERY EXAMPLES")
print("="*100)
print()

# 1. Get all simulation runs
print("1. All Simulation Runs:")
print("-" * 100)
df_runs = pd.read_sql_query("""
    SELECT run_id, week, seed, n_simulations, distribution_type, n_teams, 
           created_at
    FROM simulation_runs
    ORDER BY created_at DESC
    LIMIT 5
""", conn_proj)
print(df_runs.to_string(index=False))
print()

# 2. Get simulations for specific run
print(f"2. Simulation Results for Current Run ({run_id}):")
print("-" * 100)
df_sims_query = pd.read_sql_query(f"""
    SELECT team_id, team_name, 
           COUNT(*) as n_sims,
           AVG(total_points) as avg_points,
           MIN(total_points) as min_points,
           MAX(total_points) as max_points
    FROM monte_carlo_simulations
    WHERE run_id = '{run_id}'
    GROUP BY team_id, team_name
    ORDER BY avg_points DESC
""", conn_proj)
print(df_sims_query.to_string(index=False))
print()

# 3. Get moneyline odds for current run
print(f"3. Moneyline Odds for Current Run:")
print("-" * 100)
df_ml_query = pd.read_sql_query(f"""
    SELECT matchup, 
           team1_name, team1_ml, 
           team2_name, team2_ml
    FROM betting_odds_matchup_ml
    WHERE run_id = '{run_id}'
""", conn_proj)
if len(df_ml_query) > 0:
    print(df_ml_query.to_string(index=False))
else:
    print("No matchup data found")
print()

# 4. Get highest scorer odds
print(f"4. Highest Scorer Odds (Top 5):")
print("-" * 100)
df_high_query = pd.read_sql_query(f"""
    SELECT team_name, owner, odds, 
           ROUND(probability * 100, 2) as prob_pct,
           count
    FROM betting_odds_highest_scorer
    WHERE run_id = '{run_id}'
    ORDER BY probability DESC
    LIMIT 5
""", conn_proj)
print(df_high_query.to_string(index=False))
print()

conn_proj.close()

print("="*100)
print("Query Templates:")
print("="*100)
print()
print("Get all simulations for a specific team and week:")
print("  SELECT * FROM monte_carlo_simulations")
print("  WHERE run_id = 'YOUR_RUN_ID' AND team_id = YOUR_TEAM_ID")
print()
print("Get all betting odds for a specific week:")
print("  SELECT * FROM betting_odds_team_ou WHERE week = YOUR_WEEK")
print()
print("Compare multiple simulation runs:")
print("  SELECT sr.*, ")
print("    (SELECT AVG(total_points) FROM monte_carlo_simulations ")
print("     WHERE run_id = sr.run_id) as avg_total_points")
print("  FROM simulation_runs sr")
print()
print("="*100)
